In [5]:
import re
from cleanco import cleanco
import pandas as pd
from metaphone import doublemetaphone
import fuzzy


import numpy as np

## Sequence Matcher

### Have been using this as a sort of base-line matcher. For example, Metaphone turns words into phonetics and then this sequence matcher is used to determine similarity between those two matches.

In [ ]:
from difflib import SequenceMatcher

#Test to see : which of the two is more computationally effective and if they ever differ on normalized input

#set a correlation tolerance

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Edit Distances including Bag Distance

In [ ]:
def getEditDistance(string_one,string_two,ngrams=1,jaro_winkler=False,jaccard=False,bag=False,cosine=False,longest_common_substring= False):
    
    solutions = {}
    scores = []
    
    if jaro_winkler != False:
        solution = textdistance.JaroWinkler(qval=ngrams).normalized_similarity(string_one,string_two)
        solutions["jaro-winkler"] = solution
        
        
    if jaccard != False:
        solution = textdistance.Jaccard(qval=ngrams).normalized_similarity(string_one,string_two)
        solutions["jaccard"] = solution
    
    if bag != False:
        solution = textdistance.Bag(qval=ngrams).normalized_similarity(string_one,string_two)
        solutions["bag"] = solution
            
    if cosine != False:
        solution = textdistance.Cosine(qval=ngrams).normalized_similarity(one,two)
        solutions["cosine"] = solution
        
    if longest_common_substring != False:
        solution = textdistance.LCSStr(qval=ngrams).normalized_similarity(one,two)
        solutions["LCSS"] = solution
#     print(solutions)
    
        
    for key in solutions.keys():
        scores.append(solutions[key])
    
    return round((sum(scores)/(len(scores))),2)
        

## Levenshtein's Edit Distance:

In [ ]:

def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return "The strings are {} edits away".format(distance[row][col])

# Canonical Name Cleaner

In [49]:
def clean_company_name(company_data_frame):
    
    company_data_frame.company_name_match = company_data_frame.company_name
    # Convert to uppercase
    company_data_frame.company_name_match = company_data_frame.company_name_match.str.upper()
    # Remove commas
    company_data_frame.company_name_match = company_data_frame.company_name_match.replace(',', '')
    # Remove hyphens
    company_data_frame.company_name_match = company_data_frame.company_name_match.replace(' - ', ' ')
    # Remove text between parenthesis
    company_data_frame.company_name_match = company_data_frame.company_name_match.replace(r"\(.*\)", "")
    
    #finn: seems to strip away last bracket rather than just pulling text out
    
    
    # Remove apostrophes
    company_data_frame.company_name_match = company_data_frame.company_name_match.replace("'", "")
    #
    company_data_frame.company_name_match = company_data_frame.company_name_match.replace(' AND ', ' & ')
    # Remove spaces in the begining/end
    company_data_frame.company_name_match = company_data_frame.company_name_match.str.strip()
    # Remove business entities extensions (1)
    company_data_frame.company_name_match = company_data_frame.company_name_match.apply(
        lambda x: cleanco(x).clean_name() if type(x) == str else x)
    # Remove dots
    company_data_frame.company_name_match = company_data_frame.company_name_match.str.replace('.', '')
    # Remove business entities extensions (2) - after removing the dots
    company_data_frame.company_name_match = company_data_frame.company_name_match.apply(
        lambda x: cleanco(x).clean_name() if type(x) == str else x)
    # Specific Polish to companies
    company_data_frame.company_name_match = company_data_frame.company_name_match.str.replace('SP ZOO', '')
    # Remove European country names from supplier names
    countries = ['ALBANIA', 'ANDORRA', 'AUSTRIA', 'AZERBAIJAN', 'BELARUS', 'BELGIUM', 'BOSNIA AND HERZEGOVINA',
                 'BULGARIA', 'CROATIA', 'CYPRUS', 'CZECH REPUBLIC', 'DENMARK', 'ESTONIA', 'FAROE ISLANDS', 'FINLAND',
                 'FRANCE', 'BRITTANY', 'GERMANY', 'DEUTSCHLAND', 'GREECE', 'GUERNSEY (CHANNEL ISLANDS)', 'HUNGARY',
                 'ICELAND', 'IRELAND', 'ISLE OF MAN', 'ITALY', 'JERSEY (CHANNEL ISLANDS)', 'LATVIA', 'LIECHTENSTEIN',
                 'LITHUANIA', 'LUXEMBOURG', 'MACEDONIA', 'MALTA', 'MOLDOVA', 'MONACO', 'MONTENEGRO', 'NETHERLANDS',
                 'NEDERLAND', 'HOLLAND', 'NORWAY', 'POLAND', 'POLSKA', 'PORTUGAL', 'ROMANIA', 'RUSSIA', 'SAN MARINO',
                 'SERBIA', 'SLOVAKIA', 'SLOVENIA', 'SPAIN', 'SWEDEN', 'SWITZERLAND', 'TURKEY', 'UNITED KINGDOM', 'UK',
                 'SCOTLAND', 'WALES', 'CORNWALL', 'NORTHERN IRELAND', 'UKRAINE', 'VATICAN CITY']
    for country in countries:
        company_data_frame.company_name_match = company_data_frame.company_name_match.apply(
            lambda x: x.replace(country, '') if (type(x) == str and x.endswith(country)) else x)
    # Remove multiple whitespace
    company_data_frame.company_name_match = company_data_frame.company_name_match.replace('\s{2,}', ' ')
    return company_data_frame

# Token-based approach 

In [80]:
def token_matcher(name,name_to_match):
    
    #can include spell-checker here to see if this changes token ratio significantly.
    #ask David before doing that 
    
    token_counter = 0
    
    for word in name.split():
        if word in name_to_match.split():
            token_counter += 1
        else:
            token_counter -=1
    
    tokens_ratio = token_counter/len(name_to_match.split())
    
    print(token_counter)
    print(f"% {round(tokens_ratio,2)*100}")
    

# Trigrams Approach 

### https://stackoverflow.com/questions/46198597/python-string-matching-exactly-equal-to-postgresql-similarity-function

In [71]:



def find_ngrams(text: str, number: int=3) -> set:

    
    """
    returns a set of ngrams for the given string
    :param text: the string to find ngrams for
    :param number: the length the ngrams should be. defaults to 3 (trigrams)
    :return: set of ngram strings
    """

    if not text:
        return set()

    words = [f'  {x} ' for x in re.split(r'\W+', text.lower()) if x.strip()]

    ngrams = set()

    for word in words:
        for x in range(0, len(word) - number + 1):
            ngrams.add(word[x:x+number])

    return ngrams

In [72]:
def similarity(text1: str, text2: str, number: int=3) -> float:
    """
    Finds the similarity between 2 strings using ngrams.
    0 being completely different strings, and 1 being equal strings
    """

    ngrams1 = find_ngrams(text1, number)
    ngrams2 = find_ngrams(text2, number)

    num_unique = len(ngrams1 | ngrams2)
    num_equal = len(ngrams1 & ngrams2)

    return float(num_equal) / float(num_unique)

In [78]:
similarity("STARBUCKS SERVICES LIMITED","STARBUCKS HOLDINGS LIMITED")

0.5142857142857142

## Phonetic algorithms

### Used Soundex; Double Metaphone, with options to do it token by token or compare the whole thing using Sequence Matcher (though no option yet to do it token by token and then compare them using Sequence Matcher)

### Other phonetic algorithms that have been noted to be have more success:
Phonex
Phonix

## https://www.informit.com/articles/article.aspx?p=1848528

In [ ]:
### I've been using an apparently slower library for Metaphone 

# >>> import fuzzy
# >>> soundex = fuzzy.Soundex(4)
# >>> soundex('fuzzy')
# 'F200'
# >>> dmeta = fuzzy.DMetaphone()
# >>> dmeta('fuzzy')
# ['FS', None]
# >>> fuzzy.nysiis('fuzzy')
# 'FASY'

In [1]:
def soundex_matcher(name,name_to_match,by_tokens = False):
    
    
    soundex = fuzzy.Soundex(4)

  

   
    
    #this does a token-by-token method
    
    #this definitely seems more effective than spell checking...
    
    #this could be used within the other models for example a trigrams, consider a hierarchy processing
    
    if by_tokens == True:
    
        soundex_token_counter = 0

        for word in name.split():



            if soundex(word) in [soundex(piece) for piece in name_to_match.split()]:
                soundex_token_counter += 1
            else:
                soundex_token_counter -= 1



        soundex_tokens_ratio =  soundex_token_counter/len(name_to_match.split())



        print(f"% token soundex : {round(soundex_tokens_ratio,2)*100}")
        
    else:
        #Could use any other sequence matcher here besides similar: 
        
        def similar(a, b):
            from difflib import SequenceMatcher
            return SequenceMatcher(None, a, b).ratio()
        
        

        print(f"% non_token soundex {round(similar(soundex(name), soundex(name_to_match)),2)*100}")
       
        

In [2]:
def nysiis_matcher(name,name_to_match,by_tokens = False):
    

  
   
    
    #this does a token-by-token method
    
    #this definitely seems more effective than spell checking...
    
    #this could be used within the other models for example a trigrams, consider a hierarchy processing
    
    if by_tokens == True:
    
        nysiis_token_counter = 0

        for word in name.split():






            if fuzzy.nysiis(word) in [fuzzy.nysiis(piece) for piece in name_to_match.split()]:
                nysiis_token_counter += 1
            else:
                nysiis_token_counter -= 1



        nysiis_tokens_ratio =  nysiis_token_counter/len(name_to_match.split())



        print(f"% token nysiis : {round(nysiis_tokens_ratio,2)*100}")
        
    else:
        #Could use any other sequence matcher here besides similar: 
        
        def similar(a, b):
            from difflib import SequenceMatcher
            return SequenceMatcher(None, a, b).ratio()
        
        

        print(f"% non_token nysiis {round(similar(fuzzy.nysiis(name), fuzzy.nysiis(name_to_match)),2)*100}")
       
        

In [8]:
def metaphone_matcher(name,name_to_match,by_tokens = False):
    
    
    
    dmeta = fuzzy.DMetaphone()

  
   
    
    #this does a token-by-token method
    
    #this definitely seems more effective than spell checking...
    
    #this could be used within the other models for example a trigrams, consider a hierarchy processing
    
    if by_tokens == True:
    
        metaphone_token_counter = 0


        for word in name.split():
           

            if dmeta(word) in [dmeta(piece) for piece in name_to_match.split()]:
              
                metaphone_token_counter += 1
            else:
                
                metaphone_token_counter -= 1


         



        metaphone_tokens_ratio =  metaphone_token_counter/len(name_to_match.split())
      



        print(f"% token metaphone: {round(metaphone_tokens_ratio,2)*100}")
    
    else:
        #Could use any other sequence matcher here besides similar: 
        
        def similar(a, b):
            from difflib import SequenceMatcher
            return SequenceMatcher(None, a, b).ratio()
       

        print(f"% non_token metaphone {round(similar(dmeta(name), dmeta(name_to_match)),2)*100}")
       

## Some papers on Name Matching Algorithms

# Paper on personal-name matching

### http://users.cecs.anu.edu.au/~Peter.Christen/publications/tr-cs-06-02.pdf

## It describes more methods as well as combination of methods:
## Editex: Edit distance combined with Soundex


## The paper tests many of these algorithms with the following conclusions:
Bag Distance is the fatest, followed by n-grams.
Jaro-Winker edit distances had very strong matching capabilities
"5. The longest common sub-string technique is suitable
for unparsed names that might contain swapped words."


Favours "Calculating a similarity measure with respect to the
length of the shorter string (Overlap coefficient)"